In [ ]:
#importing all required libraries
import json
import pandas as pd
from datetime import datetime
import pytz


# Output as a CSV
Write out a csv with the following headers
- Tracking number
- Payment type (Prepaid/COD)
- Pickup Date Time in IST
- Delivery Date Time in IST
- Days taken for delivery
- Shipment weight
- Pickup Pincode, City, State
- Drop Pincode, City, State
- Number of delivery attempts needed

In [ ]:
#path to JSON file
file_path = '/content/drive/MyDrive/Swift Assignment 4 - Dataset.json'

# Reading the JSON file into a Python list
with open(file_path, 'r') as file:
    data = json.load(file)

In [ ]:
# Created an empty list to store the flattened shipment information
flattened_data = []

# Loop through each item in the data list
for track_details in data:
    # Loop through each track_detail in the "trackDetails" list within each item
    for track_detail in track_details.get("trackDetails", []):
        # Extract relevant information for each shipment
        shipment_info = {
            # Tracking number of the shipment
            "TrackingNumber": track_detail.get("trackingNumber"),

            # Payment type, extracted using the 'next' function with a condition
            "PaymentType": next(
                (
                    handling["paymentType"]
                    for handling in track_detail.get("specialHandlings", [])
                    if handling["type"] == "COD"
                ),
                None,
            ),

            # Actual pickup date and time
            "PickupDateTime": next(
                (
                    time["dateOrTimestamp"]
                    for time in track_detail.get("datesOrTimes", [])
                    if time["type"] == "ACTUAL_PICKUP"
                ),
                None,
            ),

            # Actual delivery date and time
            "DeliveryDateTime": next(
                (
                    time["dateOrTimestamp"]
                    for time in track_detail.get("datesOrTimes", [])
                    if time["type"] == "ACTUAL_DELIVERY"
                ),
                None,
            ),

            # Out for delivery date and time
            "OutForDeliveryDatetime": next(
                (
                    time["dateOrTimestamp"]
                    for time in track_detail.get("datesOrTimes", [])
                    if time["type"] == "SHIP"
                ),
                None,
            ),

            # Shipment weight, extracted from the 'shipmentWeight' field
            "ShipmentWeight": track_detail.get("shipmentWeight", {}).get("value"),

            # Pickup address details
            "PickupPincode": track_detail.get("shipperAddress", {}).get("city"),
            "PickupCity": track_detail.get("shipperAddress", {}).get("city"),
            "PickupState": track_detail.get("shipperAddress", {}).get("stateOrProvinceCode"),

            # Drop address details
            "DropPincode": track_detail.get("statusDetail", {}).get("location", {}).get("city"),
            "DropCity": track_detail.get("statusDetail", {}).get("location", {}).get("city"),
            "DropState": track_detail.get("statusDetail", {}).get("location", {}).get("stateOrProvinceCode"),
        }

        # Append the shipment information to the flattened_data list
        flattened_data.append(shipment_info)

# Create a DataFrame from the flattened_data list
df = pd.DataFrame(flattened_data)

# Convert date times to Indian Standard Time (IST)
ist = pytz.timezone('Asia/Kolkata')

df['PickupDateTime'] = pd.to_datetime(df['PickupDateTime']).dt.tz_convert(ist).dt.strftime('%Y-%m-%d %H:%M:%S %Z')
df['DeliveryDateTime'] = pd.to_datetime(df['DeliveryDateTime']).dt.tz_convert(ist).dt.strftime('%Y-%m-%d %H:%M:%S %Z')
df['OutForDeliveryDatetime'] = pd.to_datetime(df['OutForDeliveryDatetime']).dt.tz_localize('UTC').dt.tz_convert(ist).dt.strftime('%Y-%m-%d %H:%M:%S %Z')

# Display the DataFrame
display(df)


,TrackingNumber,PaymentType,PickupDateTime,DeliveryDateTime,OutForDeliveryDatetime,ShipmentWeight,PickupPincode,PickupCity,PickupState,DropPincode,DropCity,DropState
0,391128701026,OTHER,2020-03-16 15:44:00 IST,2020-03-20 13:37:00 IST,2020-03-16 05:30:00 IST,14.0,Bangalore,Bangalore,KA,Gurgaon,Gurgaon,HR
1,390901883808,None,2020-03-06 16:07:00 IST,2020-03-09 19:50:00 IST,2020-03-06 05:30:00 IST,14.0,Bangalore,Bangalore,KA,Bangalore,Bangalore,KA
2,391128749178,None,2020-03-16 15:44:00 IST,2020-03-19 15:29:00 IST,2020-03-16 05:30:00 IST,14.0,Bangalore,Bangalore,KA,Ahmedabad,Ahmedabad,GJ
3,390807986805,None,2020-03-03 16:19:00 IST,2020-03-07 14:24:00 IST,2020-03-03 05:30:00 IST,14.0,Bangalore,Bangalore,KA,New Delhi,New Delhi,DL
4,390948921190,OTHER,2020-03-09 15:12:00 IST,2020-03-13 14:44:00 IST,2020-03-09 05:30:00 IST,14.0,Bangalore,Bangalore,KA,Delhi,Delhi,DL
...,...,...,...,...,...,...,...,...,...,...,...,...
94,280439181099,None,2021-06-16 19:22:00 IST,2021-06-24 18:22:00 IST,2021-06-17 05:30:00 IST,32.0,Delhi,Delhi,DL,Chennai,Chennai,TN
95,281222569500,None,2021-07-09 14:35:00 IST,2021-07-15 18:23:00 IST,2021-07-10 05:30:00 IST,22.5,Delhi,Delhi,DL,Jammu,Jammu,JK
96,280307632740,None,2021-06-14 19:31:00 IST,2021-06-16 16:30:00 IST,2021-06-15 05:30:00 IST,2.0,Mumbai,Mumbai,MH,Pune,Pune,MH
97,280307633276,None,2021-06-14 19:31:00 IST,2021-06-16 16:30:00 IST,2021-06-15 05:30:00 IST,2.0,Mumbai,Mumbai,MH,Pune,Pune,MH


In [ ]:
# Calculate Days taken for journey completion
df['PickupDateTime'] = pd.to_datetime(df['PickupDateTime'])
df['DeliveryDateTime'] = pd.to_datetime(df['DeliveryDateTime'])
df['DaysTaken'] = (df['DeliveryDateTime'] - df['PickupDateTime']).dt.days

# Calculate Number of delivery attempts
df['OutForDeliveryDatetime'] = pd.to_datetime(df['OutForDeliveryDatetime'])
# special case where Out For Delivery and Delivered happens on same day
# Using lambda function that takes a row of the DataFrame as input and checking if the 'OutForDeliveryDatetime' is equal to 'DeliveryDateTime'. If they are equal, it assigns the value 1 (indicating a single delivery attempt); otherwise, it assigns the value 2 (indicating two delivery attempts).
df['DeliveryAttempts'] = df.apply(lambda row: 1 if row['OutForDeliveryDatetime'] == row['DeliveryDateTime'] else 2, axis=1)

In [ ]:
#testing
df

,TrackingNumber,PaymentType,PickupDateTime,DeliveryDateTime,OutForDeliveryDatetime,ShipmentWeight,PickupPincode,PickupCity,PickupState,DropPincode,DropCity,DropState,DaysTaken,DeliveryAttempts
0,391128701026,OTHER,2020-03-16 15:44:00,2020-03-20 13:37:00,2020-03-16 05:30:00,14.0,Bangalore,Bangalore,KA,Gurgaon,Gurgaon,HR,3,2
1,390901883808,None,2020-03-06 16:07:00,2020-03-09 19:50:00,2020-03-06 05:30:00,14.0,Bangalore,Bangalore,KA,Bangalore,Bangalore,KA,3,2
2,391128749178,None,2020-03-16 15:44:00,2020-03-19 15:29:00,2020-03-16 05:30:00,14.0,Bangalore,Bangalore,KA,Ahmedabad,Ahmedabad,GJ,2,2
3,390807986805,None,2020-03-03 16:19:00,2020-03-07 14:24:00,2020-03-03 05:30:00,14.0,Bangalore,Bangalore,KA,New Delhi,New Delhi,DL,3,2
4,390948921190,OTHER,2020-03-09 15:12:00,2020-03-13 14:44:00,2020-03-09 05:30:00,14.0,Bangalore,Bangalore,KA,Delhi,Delhi,DL,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,280439181099,None,2021-06-16 19:22:00,2021-06-24 18:22:00,2021-06-17 05:30:00,32.0,Delhi,Delhi,DL,Chennai,Chennai,TN,7,2
95,281222569500,None,2021-07-09 14:35:00,2021-07-15 18:23:00,2021-07-10 05:30:00,22.5,Delhi,Delhi,DL,Jammu,Jammu,JK,6,2
96,280307632740,None,2021-06-14 19:31:00,2021-06-16 16:30:00,2021-06-15 05:30:00,2.0,Mumbai,Mumbai,MH,Pune,Pune,MH,1,2
97,280307633276,None,2021-06-14 19:31:00,2021-06-16 16:30:00,2021-06-15 05:30:00,2.0,Mumbai,Mumbai,MH,Pune,Pune,MH,1,2


In [ ]:
# Saving the output to a CSV file
df.to_csv('output.csv', index=False)

#**Output Summary Statistics as a CSV**

Output a summary having:
- Mean/Median/Mode of days taken for delivery
- Mean/Median/Mode of delivery attemps

In [ ]:
# Calculate Mean/Median/Mode of DaysTaken and DeliveryAttempts
summary_statistics = {
    'DaysTaken_mean': df['DaysTaken'].mean(),
    'DaysTaken_median': df['DaysTaken'].median(),
    'DaysTaken_mode': df['DaysTaken'].mode().iloc[0],
    'DeliveryAttempts_mean': df['DeliveryAttempts'].mean(),
    'DeliveryAttempts_median': df['DeliveryAttempts'].median(),
    'DeliveryAttempts_mode': df['DeliveryAttempts'].mode().iloc[0],
}

# Convert summary statistics to DataFrame
summary_df = pd.DataFrame([summary_statistics])

#testing
summary_df

,DaysTaken_mean,DaysTaken_median,DaysTaken_mode,DeliveryAttempts_mean,DeliveryAttempts_median,DeliveryAttempts_mode
0,3.262626,3.0,3,2.0,2.0,2


In [ ]:
# Saving the summary statistics to a CSV file
summary_df.to_csv('summary_statistics.csv', index=False)